In [ ]:
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt

# Define the bounding box: (north, south, east, west)
bbox = (41.86843, 41.86726, 12.535509, 12.534124)  # Example coordinates for a small area in Rome

# List key-value pairs for tags (e.g., 'building': True)
tags = {'building': True}

# Retrieve buildings data from OpenStreetMap for the bounding box
buildings = ox.features_from_bbox(bbox, tags)


/home/emaximus/workspace/lsasaf_data_access/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:267: UserWarning: This area is 3,743 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [ ]:

# Clean up the dataframe by removing unnecessary columns
buildings = buildings.drop(['nodes', 'building'], axis=1, errors='ignore')

# Calculate the centroid of each building's geometry
buildings["centroid_lat"] = buildings["geometry"].centroid.y
buildings["centroid_lng"] = buildings["geometry"].centroid.x

# Convert geometry to a projection suited for area calculation (EPSG:3395 is CEA - equal area)
buildings = buildings.to_crs(epsg=3395)

# Calculate the area of each building in square meters
buildings["area"] = buildings["geometry"].area

# Optionally drop the 'geometry' column if you don't need it
# buildings = buildings.drop('geometry', axis=1)

# Output building data and save to CSV
# print(buildings)
# buildings.to_csv('buildings_in_bbox.csv')

# Calculate the sum of areas and count of buildings
total_area = buildings["area"].sum()
building_count = buildings.shape[0]
avg_area = total_area / building_count

print(f"Total buildings: {building_count}")
print(f"Sum of building areas: {total_area:.2f} sqm or {total_area / 10**6:.2f} km²")
print(f"Average building area: {avg_area:.2f} sqm")

# Display a histogram of building areas, excluding extreme outliers
quantile = 0.99
trimmed_data = buildings[(buildings["area"] > buildings["area"].quantile(1-quantile)) &
                         (buildings["area"] < buildings["area"].quantile(quantile))]

plt.hist(trimmed_data["area"], bins=100)
plt.xlabel("Building Area (sqm)")
plt.ylabel("Frequency")
plt.title("Histogram of Building Areas (Trimmed)")
plt.show()
